# Mentoria DiploDatos FAMAF

## Reducción de Emisiones Contaminantes por el Uso de Biocombustibles en Transporte de Cargas y Pasajeros

### Práctico de Análisis y Visualización

In [1]:
# Data Tools
import pandas as pd
import numpy as np

# Graphs
import matplotlib.pyplot as plt
import plotly.offline as pof
import plotly.graph_objects as pgo
import seaborn as sns
from datetime import datetime

## Configuration
import configparser

In [2]:
config = configparser.ConfigParser()
config.read('config.ini')
conf = config['dtondo']

In [3]:
#import warnings
#warnings.filterwarnings("ignore")

In [4]:
import pymysql as sql
##import sqlalchemy as db

## Datos propios
_Tablas del Dataset :_
- **Usuarios:** registra datos de los Operadores que realizan las transacciones de despacho
- **Vehiculos:** registra datos de las Unidades de Consumo o Vehículos
- **Productos:** registra datos de los tipos de combustibles
- **Equipos:** registra datos del Gateway que conecta cada Centro Operativo
- **Tanques:** registra datos de los tanques de almacenamiento en cada Centro Operativo
- **Bombas:** registra datos de las Picos de Despacho de combustibles
- **Transacciones (mensuales, desde el 2018):** registro de los despachos de combustible en cada Centro Operativo a cada vehículo
- **Historial (mensuales, desde el 2018):** registro de series temporales de inventarios de combustibles en cada tanque de almacenamiento


### Conexión a DB utilizando PyMySQL

In [5]:
## Database Connection using MySQL
db = sql.connect(
  host=conf['DB_HOST'],
  user=conf['DB_USER'],
  passwd=conf['DB_PASS'],
  database=conf['DB_NAME']
)
dbc = db.cursor()

In [6]:
# Listado de todas las tablas de la base de datos
dbc.execute("show tables")
tablas  = dbc.fetchall()
tablas

(('fs_asignacion_producto',),
 ('fs_bombas',),
 ('fs_equipo',),
 ('fs_tanques',),
 ('fs_usuarios_fuelsentry',),
 ('fs_vehiculos',),
 ('sis_historial_2018_1',),
 ('sis_historial_2018_10',),
 ('sis_historial_2018_11',),
 ('sis_historial_2018_12',),
 ('sis_historial_2018_2',),
 ('sis_historial_2018_3',),
 ('sis_historial_2018_4',),
 ('sis_historial_2018_5',),
 ('sis_historial_2018_6',),
 ('sis_historial_2018_7',),
 ('sis_historial_2018_8',),
 ('sis_historial_2018_9',),
 ('sis_historial_2019_1',),
 ('sis_historial_2019_10',),
 ('sis_historial_2019_11',),
 ('sis_historial_2019_12',),
 ('sis_historial_2019_2',),
 ('sis_historial_2019_3',),
 ('sis_historial_2019_4',),
 ('sis_historial_2019_5',),
 ('sis_historial_2019_6',),
 ('sis_historial_2019_7',),
 ('sis_historial_2019_8',),
 ('sis_historial_2019_9',),
 ('sis_historial_2020_1',),
 ('sis_historial_2020_2',),
 ('sis_historial_2020_3',),
 ('sis_historial_2020_4',),
 ('sis_historial_2020_5',),
 ('sis_transa_2018_1',),
 ('sis_transa_2018_10',),

In [7]:
# Recuperar los 10 primeros registros de una tabla de la base de datos
query = "SELECT * FROM fs_bombas;"
dbc.execute(query)
registros  = dbc.fetchall()
registros[0]

(1,
 333333,
 '01',
 '0',
 1,
 '0031731.441',
 datetime.date(2012, 7, 19),
 '0000',
 '2.0',
 '1',
 '0000')

### Conexión a DB usando Engine del ORM SQLAlchemy

In [8]:
# Conexión a la base de datos (para MySQL)
from sqlalchemy import create_engine

In [9]:
# Crear motor de conexión sqlalchemy
path_conexion = 'mysql+pymysql://{}:{}@{}:{}/{}'.format(conf['DB_USER'], conf['DB_PASS'], conf['DB_HOST'], conf['DB_PORT'], conf['DB_NAME'])
dbconn = create_engine(path_conexion)

Para este primer práctico estaremos trabajando solo con algunos datasets. En el segundo práctico se incorporará otros datasets, que requieren de mayor limpieza y manipulacion de los datos.

In [10]:
# Obtención de dataframe 'fs_asignacion_producto', 'fs_bombas', 'fs_equipo', 'fs_tanques', 'fs_usuarios_fuelsentry', 'fs_vehiculos'
df_p = pd.read_sql_query("SELECT * FROM fs_asignacion_producto;", dbconn)
df_t = pd.read_sql_query("SELECT * FROM fs_tanques;", dbconn)
df_m = pd.read_sql_query("SELECT * FROM fs_usuarios_fuelsentry;", dbconn)
df_v = pd.read_sql_query("SELECT * FROM fs_vehiculos;", dbconn)

format_string = "%Y-%m-%d %H:%M:%S"
df_b = pd.read_sql_query("SELECT * FROM fs_bombas;", dbconn, parse_dates={'fecha_reinicio': format_string})

In [ ]:
#pd.set_option('display.max_columns', 50)

In [19]:
[ x.shape for x in [df_p, df_t, df_m, df_v, df_b]]

[(2672, 7), (1561, 7), (13623, 7), (50627, 13), (1438, 11)]

## Test de Velocidad cargando datos

In [ ]:
%%time
df_h1 = pd.read_sql_query('SELECT * FROM sis_historial_2020_4', dbconn)

In [ ]:
%%time
df_h2 = pd.read_csv('../../dataset/sis_historial_2020_4.csv')

In [ ]:
%%time
df_h3 = pd.read_sql_table('sis_historial_2020_4', dbconn)

## REVISION DE DATASET
### Dataset Productos:

Contiene información relacionada con los productos de combustible.

Descripción de las columnas:
- 'id_equipo': ID del dispositivo IoT
- 'producto': código del producto
- 'nombre_producto': nombre del producto
- 'codigo': 
- 'precio_litro': precio del producto (en $/l)
- 'coef_var_vol': coeficiente de variación del volumen del producto combustible (en g/ml/°C)
- 'density': densidad del producto (en g/ml)

Densidad (ASTM D 4052): densidad relativa del combustible medido (en g/ml o kg/m3) a la temperatura estándar de 15 °C.

,id_equipo,producto,nombre_producto,codigo,precio_litro,coef_var_vol,density
1102,372,0,GAS OIL,GAS OIL,57.5,0.001,0.84
1688,477,4,Comun,Comun,1.0,0.001,1.00
2586,465,4,Gas Oil,None,0.0,10.000,1.00


In [64]:
#print(f'Dimensión del dataset (filas, columnas): {df_p.shape}\n')
df_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2672 entries, 0 to 2671
Data columns (total 7 columns):
id_equipo          2672 non-null int64
producto           2672 non-null object
nombre_producto    2672 non-null object
codigo             2526 non-null object
precio_litro       2672 non-null float64
coef_var_vol       2672 non-null float64
density            249 non-null float64
dtypes: float64(3), int64(1), object(3)
memory usage: 146.2+ KB


In [28]:
# Valores o categorias unicas para cada columna
# for i in df_p.columns:
#     print(f'{i}: {df_p[i].nunique()}')
df_p.nunique()

id_equipo          702
producto            10
nombre_producto    142
codigo             143
precio_litro        76
coef_var_vol         5
density              3
dtype: int64

In [82]:
## Descriptor rapido por columna
df_p.describe()

,id_equipo,precio_litro,coef_var_vol,density
count,2672.000000,2672.000000,2672.000000,249.000000
mean,31287.472680,1.219868,0.004741,0.011406
std,162581.386309,6.563341,0.193436,0.103822
min,1.000000,-48.000000,0.000000,0.000000
25%,246.000000,0.000000,0.001000,0.000000
50%,438.000000,0.000000,0.001000,0.000000
75%,668.000000,0.000000,0.001000,0.000000
max,999999.000000,62.500000,10.000000,1.000000


In [83]:
## Descriptor de columnas no numericas
df_p.describe(include = ['object'])

,producto,nombre_producto,codigo
count,2672,2672,2526
unique,10,142,143
top,0,Product 4,Product 4
freq,701,550,517


In [80]:
## Valores de densidad
df_p[pd.notna(df_p.density) & df_p.density != 0.0 ]

,id_equipo,producto,nombre_producto,codigo,precio_litro,coef_var_vol,density
1102,372,0,GAS OIL,GAS OIL,57.5,0.001,0.84
1688,477,4,Comun,Comun,1.0,0.001,1.00
2586,465,4,Gas Oil,None,0.0,10.000,1.00


**ONLY 3 VALUES WITH DENSITY DATA!!!**, this column could be dropped or filled by product.

In [70]:
print( pd.notna(df_p.coef_var_vol).size / len(df_p) )

1.0


The Column "COEF_VAR_VOL" has not empty values.

In [71]:
len(df_p.coef_var_vol != 0)

2672

### Dataset de bombas:

Contiene información de las bombas de suministro de combustible.

--> Un punto de suministro del combustible posee uno o varios equipos (dispositivo IoT) 

Descripción de las columnas:
- 'id_bomba': ID de la bomba
- 'id_equipo': ID del dispositivo IoT
- 'bomba': 
- 'producto': tipo de combustible que suministra la bomba
- 'id_tanque': ID del tanque
- 'totalizador': litros suministrados a la fecha de reinicio
- 'fecha_reinicio': fecha de reinicio de la bomba
- 'pulsos_litro': litros que suministra la bomba (por pulso)
- 'tiempo_interrump': tiempo de interrupción del suministro de combustible de la bomba (en segundos)
- 'habilitacion': 
- 'rampa_de_parada'

In [79]:
print(f'Dimensión del dataset (filas, columnas): {df_b.shape}\n')
print(f'{df_b.info()}\n')
df_b

Dimensión del dataset (filas, columnas): (1438, 11)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1438 entries, 0 to 1437
Data columns (total 11 columns):
id_bomba            1438 non-null int64
id_equipo           1438 non-null int64
bomba               1438 non-null object
producto            1438 non-null object
id_tanque           1438 non-null int64
totalizador         1438 non-null object
fecha_reinicio      1435 non-null datetime64[ns]
pulsos_litro        1438 non-null object
tiempo_interrump    1438 non-null object
habilitacion        1438 non-null object
rampa_de_parada     1438 non-null object
dtypes: datetime64[ns](1), int64(3), object(7)
memory usage: 123.7+ KB
None



,id_bomba,id_equipo,bomba,producto,id_tanque,totalizador,fecha_reinicio,pulsos_litro,tiempo_interrump,habilitacion,rampa_de_parada
0,1,333333,01,0,1,0031731.441,2012-07-19,0000,2.0,1,0000
1,2,333333,02,0,1,1160601.525,2012-07-19,0000,2.0,1,0000
2,11417,770,02,1,101856,0000000.000,2018-07-05,0026.3158,020,1,0000
3,5,444444,01,2,4,0651995.027,2010-12-01,35.0,888,1,0000
4,6,444444,02,2,4,0005232.793,2010-12-01,35.5,2.0,1,0000
...,...,...,...,...,...,...,...,...,...,...,...
1433,11498,661,01,1,101495,0000000.519,2018-07-05,0000.0000,120,1,0000
1434,11505,820,01,1,102038,2463692.430,2016-08-24,0000.0000,200,1,0000
1435,11506,820,02,1,102039,2564723.192,2016-08-24,0000.0000,120,1,0000
1436,11507,820,03,2,102040,1097025.715,2016-08-24,0000.0000,120,1,0000


In [72]:
# Valores o categorias unicas para cada columna
df_b.nunique()

id_bomba            1438
id_equipo            610
bomba                 28
producto              11
id_tanque            861
totalizador          869
fecha_reinicio       179
pulsos_litro         245
tiempo_interrump      29
habilitacion           7
rampa_de_parada        7
dtype: int64

### Dataset de tanques:

Contiene información de los tanques de combustible.

--> Una empresa puede tener más centro operativo (equipo)

--> Un equipo representa un centro operativo

--> Un centro operativo puede tener multiples tanques y bombas (picos)

Descripción de las columnas:
- 'id_tanque': ID del tanque
- 'id_equipo': ID del dispositivo IoT 
- 'tanque': tipo de tanque
- 'producto': producto almacenado en el tanque 
- 'capacidad': capacidad del tanque en litros
- 'log_interval': intervalo de tiempo en el cual se registra la medición del volumen del contenido del tanque --> historial del volumen del tanque
- 'nivel_alarma': nivel de contenido del tanque para disparar alarma

In [86]:
df_t

,id_tanque,id_equipo,tanque,producto,capacidad,log_interval,nivel_alarma
0,101525,669,B,1,0010000.1,0030,0
1,101833,764,5,1,0008000.0,0030,000003200
2,101496,662,1,0,0050000.0,0030,0
3,101495,661,1,0,0025000.0,0030,0
4,101522,669,8,1,0010000.1,0030,0
...,...,...,...,...,...,...,...
1556,102053,446,1,2,0010000.1,0010,0
1557,102054,446,2,2,0061481.0,0060,0
1558,102055,446,3,2,0010000.1,0010,0
1559,102047,821,1,1,0010000.0,0060,0


In [84]:
df_t.nunique()

id_tanque       1561
id_equipo        638
tanque            18
producto          10
capacidad        158
log_interval      18
nivel_alarma     102
dtype: int64

In [87]:
df_t.dtypes

id_tanque        int64
id_equipo        int64
tanque          object
producto        object
capacidad       object
log_interval    object
nivel_alarma    object
dtype: object

In [89]:
df_t.describe(include=['int64', 'object'])

,id_tanque,id_equipo,tanque,producto,capacidad,log_interval,nivel_alarma
count,1561.000000,1561.000000,1561,1561,1561,1561,1561
unique,NaN,NaN,18,10,158,18,102
top,NaN,NaN,1,1,0050000.0,0030,0
freq,NaN,NaN,636,771,204,1276,1033
mean,47550.973094,1716.206278,NaN,NaN,NaN,NaN,NaN
std,50362.698081,35769.926657,NaN,NaN,NaN,NaN,NaN
min,43.000000,1.000000,NaN,NaN,NaN,NaN,NaN
25%,552.000000,248.000000,NaN,NaN,NaN,NaN,NaN
50%,1123.000000,396.000000,NaN,NaN,NaN,NaN,NaN
75%,101499.000000,612.000000,NaN,NaN,NaN,NaN,NaN


### Dataset de usuarios:

Contiene información de los usuarios de las bombas de combustible.

Descripción de las columnas:
- 'id_usuario_fuelsentry': ID de registro del usuario
- 'id_equipo':  ID del dispositivo IoT
- 'usuario_fuelsentry': código del usuario de la bomba
- 'departamento': 
- 'codigo': 
- 'totalizador': litros totales suministrados por el usuario de la bomba
- 'cargas_totales': número total de cargas sumnistradas por el usuario

In [90]:
df_m

,id_usuario_fuelsentry,id_equipo,usuario_fuelsentry,departamento,codigo,totalizador,cargas_totales
0,2,333333,0001,0000,0000,00323.152,001
1,3,333333,1315,0001,2012,01655.590,009
2,4,333333,1312,0001,4468,01433.429,006
3,5,333333,1298,0001,2609,00412.834,002
4,6,333333,0818,0001,1978,00713.023,003
...,...,...,...,...,...,...,...
13618,16390,678,0590,0001,0000,00000.000,000
13619,16391,790,0590,0001,0000,00000.000,000
13620,16392,635,0591,0001,0000,00000.000,000
13621,16393,678,0591,0001,0000,00000.000,000


In [92]:
df_m.nunique()

id_usuario_fuelsentry    13623
id_equipo                  509
usuario_fuelsentry        3527
departamento                64
codigo                     594
totalizador               4523
cargas_totales             549
dtype: int64

### Dataset de Vehiculos

In [93]:
df_v

,id_vehiculo,id_equipo,vehiculo,departamento,limite,odometro_inicio,odometro_fin,cargas_max_dia,autorizacion,cantidad_total,cargas_hoy,cargas_hasta_hoy,ultima_fecha
0,2,333333,MDAwMDAx,0001,9,193958,193958,99,0,00000.000,01,000,2018-12-20
1,3,333333,MDAwMjM5,0001,9,079345,079345,99,0,00000.000,02,000,2018-12-20
2,4,333333,MDAwMjcw,0001,9,842409,842409,99,0,00000.000,01,000,2018-12-20
3,5,333333,MDAwMjUy,0001,9,287596,287596,99,0,00000.000,02,000,2018-12-20
4,6,333333,MDAwMjM1,0001,9,235878,235878,99,0,00000.000,01,000,2018-12-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50622,101045,515,MDI1MTIy,0001,9,000000,000000,00,0,00000.000,00,000,2020-06-13
50623,101043,372,MDAwNTA3,0008,9,000000,000000,00,0,00093.547,01,001,2020-06-12
50624,101044,792,MDAwMDk5,0000,7,000000,000000,01,0,00000.000,00,000,2020-06-12
50625,101048,625,MzAwMDA1,0001,9,000000,000000,00,0,00000.000,00,000,2020-06-13


In [94]:
df_v.dtypes

id_vehiculo          int64
id_equipo            int64
vehiculo            object
departamento        object
limite              object
odometro_inicio     object
odometro_fin        object
cargas_max_dia      object
autorizacion        object
cantidad_total      object
cargas_hoy          object
cargas_hasta_hoy    object
ultima_fecha        object
dtype: object

In [95]:
df_v.describe(include=['int64', 'object'])

,id_vehiculo,id_equipo,vehiculo,departamento,limite,odometro_inicio,odometro_fin,cargas_max_dia,autorizacion,cantidad_total,cargas_hoy,cargas_hasta_hoy,ultima_fecha
count,50627.000000,50627.000000,50627,50627,50627,50627,50627,50627,50627,50627,50627,50627,50627
unique,NaN,NaN,8111,225,14,9879,10472,24,13,13937,47,525,828
top,NaN,NaN,MDAwMDAx,0001,9,000000,000000,00,0,00000.000,00,000,2019-04-09
freq,NaN,NaN,426,18759,23274,36982,37001,40997,46246,32465,25008,35347,18464
mean,66999.764177,8177.155016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,21436.618465,78681.218017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,2.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,50313.500000,279.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,68652.000000,488.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,85335.500000,591.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Datos públicos:
### Bioetanol: producción y ventas (total país)
- Producción y ventas de bioetanol por mes en base a maíz y caña de azúcar (en metros cúbicos).
- Frecuencia de actualización: Mensualmente
- Último cambio: 15 de mayo de 2018

In [105]:
url_bioetanol_nacional = 'http://datos.minem.gob.ar/dataset/5ce77ad1-c729-42cd-a8b5-2407de005e5b/resource/fd142c49-fa73-4e21-be1f-a10d6d67d05d/download/bioetanol-serie-produccion-y-ventas-total-pais.csv'
df_bioetanol=pd.read_csv(url_bioetanol_nacional)

In [106]:
df_bioetanol

,mes,unidades,produccion_total,ventas_totales,produccion_base_maiz,ventas_bioetanol_maiz,produccion_base_cana_de_azucar,ventas_bioetanol_cana_de_azucar
0,11/1/2009,metros cúbicos,858.00,0.00,0.00,0.00,858.00,0.00
1,12/1/2009,metros cúbicos,22439.00,2664.00,0.00,0.00,22439.00,2664.00
2,1/1/2010,metros cúbicos,771.00,6188.00,0.00,0.00,771.00,6188.00
3,2/1/2010,metros cúbicos,4791.00,6718.00,0.00,0.00,4791.00,6718.00
4,3/1/2010,metros cúbicos,4445.00,9585.00,0.00,0.00,4445.00,9585.00
...,...,...,...,...,...,...,...,...
96,11/1/2017,metros cúbicos,96565.42,94110.56,47841.45,49530.77,48723.97,44579.79
97,12/1/2017,metros cúbicos,87520.90,100039.60,51159.89,57177.65,36361.01,42861.95
98,1/1/2018,metros cúbicos,83694.36,98469.40,50443.30,54715.66,33251.06,43753.74
99,2/1/2018,metros cúbicos,80920.94,88478.52,45592.72,45030.36,35328.22,43448.16


In [116]:
df_bioetanol['mes'] = pd.to_datetime(df_bioetanol['mes'])
df_bioetanol.mes.max()

Timestamp('2018-03-01 00:00:00')

### Biodiesel: producción, ventas y exportaciones, total país
- Producción, ventas y exportaciones de biodiesel por mes en toneladas.
- Frecuencia de actualización: Mensualmente
- Último cambio: 15 de mayo de 2018

In [114]:
url_biodiesel_nacional = 'http://datos.minem.gob.ar/dataset/5ce77ad1-c729-42cd-a8b5-2407de005e5b/resource/4e04bc74-8625-412c-acc2-48412f2509b4/download/biodiesel-serie-produccion-ventas-y-expo.csv'
df_biodiesel = pd.read_csv(url_biodiesel_nacional)
df_biodiesel
# Leer CSV

,mes,unidades,produccion_total,ventas_al_corte,otras_ventas_al_mercado_interno,exportaciones
0,1/1/2008,ton,47251.00,0.00,1.00,43599.00
1,2/1/2008,ton,38747.00,0.00,0.00,25147.00
2,3/1/2008,ton,42095.00,0.00,264.00,52150.00
3,4/1/2008,ton,43881.00,0.00,0.00,55907.00
4,5/1/2008,ton,51679.00,0.00,0.00,42214.00
...,...,...,...,...,...,...
118,11/1/2017,ton,302618.48,102110.27,57.56,185000.00
119,12/1/2017,ton,265727.76,105923.82,28.12,170790.77
120,1/1/2018,ton,244128.16,82481.54,59.76,213909.23
121,2/1/2018,ton,157007.22,83808.80,29.84,40000.00


In [118]:
df_biodiesel['mes'] = pd.to_datetime(df_biodiesel['mes'])
df_biodiesel.mes.max()

Timestamp('2018-03-01 00:00:00')

## Análisis a desarrollar:

Indicaciones previas: 
- Para cada planteamiento (excepto el 1), realizar un análisis de los resultados obtenidos y justificar por qué empleó determinado cálculo.

- Para los planteamientos donde utilices gráficos, responder: ¿Qué tipo de gráfico es el recomendado? ¿Por qué? Construya el gráfico y elabore una conclusión del mismo.

Nota: Cuidar los aspectos relevantes de un gráfico (título, nombre de las variables para cada eje, escala de valores en los ejes, colores, fuente de los datos, leyenda, valores a resaltar, etc)

1- Seleccionar y crear una lista de 5 variables númericas. Comenta si son de tipo discreto o continuo, y por qué.

2- Determinar algunas medidas estadísticas para las variables seleccionadas.

3- Obtenga el número registros en el dataset para cada producto ('producto' de df_bombas). Comente sobre los valores obtenidos.

4- Obtener un gráfico para visualizar el comportamiento de los valores de capacidad de los tanques de combustibles. (df_tanques)

5- Obtener un gráfico que muestre y compare el comportamiento del coeficiente de variación de los distintos productos (df_productos). ¿Qué haría para mejorar el gráfico?

6- Analizar la distribución de los litros totales y la cantidad de cargas totales que suministraron los usuarios ('totalizador' y 'cargas_totales' del df_usuarios)

#### Emplear los datos públicos del Bioetanol y Biodiesel: producción y ventas (total país), en las últimos planteamientos:

7- Obtener las gráficas de series temporales de la producción de bioetanol en base a caña de azúcar, a maíz y la total, así también para la producción total de biodiesel.

8- Comparar la distribución de producción total de biodiesel y bioetanol para cada mes y año. ¿En qué años y meses los consumos son más consistentes?
--> Emplear: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html#pandas.pivot_table

9- Obtener un mapa de calor por mes y año, para los valores medios de  producción total de biodiesel y bioetanol.